In [11]:
import opstrat as op
import pandas as pd
import numpy as np
import random
from scipy.stats import norm
import matplotlib.pyplot as plt
import math
import seaborn as sns

import json
with open('config.json') as config_file:
     data = json.load(config_file)
print(data)

last_price          = data['Current_Price']
annual_drift        = data['Annual_Drift']
annual_volatility   = data['Annual_Volatiltiy']
time_period         = data['Time_Period']
simu                = data['No_of_days']
r                   = data['rate_return']
sigma               = data['sigma']
price               = data['Current_Price']

df = pd.DataFrame()

last_price_list = [last_price]
diff_list       = []
tenor_list      = [0.083333333333333333]
count           = 0
count_list      = []
prob_call_list  = []
prob_put_list   = []
k1              = int(round(price,-2))+600
p1              = int(round(price,-2))-600

Trading_Days = 252
Expiry_Days = 21


Daily_volatility = sigma*math.sqrt(Expiry_Days/Trading_Days)


while count < simu:
    count = count + 1


    sigma      = Daily_volatility*last_price
    prob_call = norm.pdf((k1-last_price)/sigma)
    prob_put = norm.pdf((p1-last_price)/sigma)

    difference = last_price*annual_drift*time_period + last_price*annual_volatility*(time_period**(0.5))*norm.ppf(random.random())

    last_price = last_price + difference
    tenor      = ((21-count)/(21*12))
    
    if prob_call>=0.30 or prob_put >= 0.30:
         break
        


    last_price_list.append(last_price)
    diff_list.append(difference)
    count_list.append(count)
    tenor_list.append(tenor)
    prob_call_list.append(prob_call)
    prob_put_list.append(prob_put)
    

last_price_list.pop(-1)
tenor_list.pop(-1)

df['Count']         = count_list
df['Price']         = last_price_list
df['Change_price']  = diff_list
df['tenor']         = tenor_list
df['Sigma']         = Daily_volatility*df['Price']
df['Prob_call']     = prob_call_list
df['Prob_put']      = prob_put_list

{'Current_Price': 17689, 'Annual_Drift': 0.07, 'Annual_Volatiltiy': 0.15, 'Time_Period': 0.003968254, 'rate_return': 0.069, 'sigma': 0.1279, 'No_of_days': 21}


In [12]:
df

,Count,Price,Change_price,tenor,Sigma,Prob_call,Prob_put
0,1,17689.000000,-72.167640,0.083333,653.105293,0.257548,0.265643
1,2,17616.832360,4.549782,0.079365,650.440752,0.229806,0.290944
2,3,17621.382142,-214.332522,0.075397,650.608737,0.231560,0.289373


In [13]:
from math import log, sqrt, pi, exp
from scipy.stats import norm
from datetime import datetime, date
import numpy as np
import pandas as pd
from pandas import DataFrame
import math


sigma               = data['sigma']
list1 = [int(round(price, -2))+900,int(round(price, -2))+600]
list2 = [int(round(price, -2))-800,int(round(price, -2))-600]
datalist = dict()
for k,p in zip(list1,list2):
    call_option     =   []
    call_thetalist  =   []
    call_gammalist  =   []
    call_deltalist  =   []
    call_rholist    =   []
    call_vegalist   =   [] 
    #Prob_list       =   []
    put_option      =   []
    put_thetalist   =   []
    put_gammalist   =   []
    put_deltalist   =   []
    put_rholist     =   []
    put_vegalist    =   []

    for last_price,T in zip(df['Price'],df['tenor']):

        d1         = (math.log(last_price/k)+(r+((sigma**2)/2))*T)/(sigma*(T**(0.5)))
        d2         = d1-sigma*(T**0.5)
        N_d1       = norm.cdf(d1)
        N_d2       = norm.cdf(d2)
        N_d1_      = 1-norm.cdf(d1)
        N_d2_      = 1-norm.cdf(d2)
        call       = last_price*N_d1-k*math.exp(-r*T)*N_d2
        #put        = k*(math.exp(-r*T))*(1-N_d2)-last_price*(1-N_d1)
        call_delta = norm.cdf(d1)
        call_gamma = norm.pdf(d1)/(last_price*sigma*sqrt(T))
        call_vega  = (last_price*norm.pdf(d1)*sqrt(T))
        call_theta = ((-(last_price*norm.pdf(d1)*sigma)/(2*sqrt(T))) - r*k*exp(-r*T)*norm.cdf(d2))
        call_rho   = (k*T*exp(-r*T)*norm.cdf(d2))
        
        
        #Prob = norm.pdf((k-last_price)/sig)


        #Prob_list.append(Prob)
        call_option.append(call)
        call_thetalist.append(call_theta)
        call_gammalist.append(call_gamma)
        call_deltalist.append(call_delta)
        call_rholist.append(call_rho)
        call_vegalist.append(call_vega)

        #datalist['Prob_call_' + str(k)] =   Prob_list
        datalist['CE_'+str(k)]          =   call_option
        datalist['theta_CE'+str(k)]     =   call_thetalist
        datalist['gamma_CE'+str(k)]     =   call_gammalist
        datalist['delta_CE'+str(k)]     =   call_deltalist
        datalist['rho_CE'+str(k)]       =   call_rholist
        datalist['vega_CE'+str(k)]      =   call_vegalist


        d1         = (math.log(last_price/p)+(r+((sigma**2)/2))*T)/(sigma*(T**(0.5)))
        d2         = d1-sigma*(T**0.5)
        N_d1       = norm.cdf(d1)
        N_d2       = norm.cdf(d2)
        N_d1_      = 1-norm.cdf(d1)
        N_d2_      = 1-norm.cdf(d2)
        call       = last_price*N_d1-p*math.exp(-r*T)*N_d2
        put        = p*(math.exp(-r*T))*(1-N_d2)-last_price*(1-N_d1)
        put_delta  = -norm.cdf(-d1)
        put_gamma  = norm.pdf(d1)/(last_price*sigma*sqrt(T))
        put_vega   = (last_price*norm.pdf(d1)*sqrt(T))
        put_theta  = (-(last_price*norm.pdf(d1)*sigma)/(2*sqrt(T)) + r*p*exp(-r*T)*norm.cdf(-d2))
        put_rho    = (-p*T*exp(-r*T)*norm.cdf(-d2))
        
        put_option.append(put)
        put_thetalist.append(put_theta)
        put_gammalist.append(put_gamma)
        put_deltalist.append(put_delta)
        put_rholist.append(put_rho)
        put_vegalist.append(put_vega)

        datalist['PE_'+str(p)]         = put_option
        datalist['theta_PE_'+str(p)]   = put_thetalist
        datalist['gamma_PE_'+str(p)]   = put_gammalist
        datalist['delta_PE_'+str(p)]   = put_deltalist
        datalist['rho_PE_'+str(p)]     = put_rholist
        datalist['vega_PE_'+str(p)]    = put_vegalist
        
df1 = pd.DataFrame(datalist)

df = pd.concat([df,df1],axis=1)

In [14]:
#Changing the signs of greeks according to options selling and buying
df['PE_'+str((round(price, -2))-800)] = - df['PE_'+str((round(price, -2))-800)]
df['CE_'+str((round(price, -2))+900)] = - df['CE_'+str((round(price, -2))+900)]


put_sell = ['theta_PE_','gamma_PE_','delta_PE_','rho_PE_','vega_PE_']
for data in put_sell:
    df[data+str((round(price, -2))-600)] = - df[data+str((round(price, -2))-600)]

call_sell = ['theta_CE','gamma_CE','delta_CE','rho_CE','vega_CE']
for data in call_sell:
    df[data+str((round(price, -2))+600)] = - df[data+str((round(price, -2))+600)]




#Calculating the total premium paid or recieved 
filter_col = [col for col in list(df) if col.startswith(('PE_','CE_'))]
df['Total_Prem'] = 0 # initialize value, otherwise throws KeyErrorx
for d in df[filter_col]:
    df['Total_Prem'] = df[filter_col].sum(axis=1)   # Total_PP = total put premium 


#Calculating the total Profit and Loss

df['Total_P&L'] = df['Total_Prem'][0] - df['Total_Prem']


#Calculating greeks for overall data

list1 = ['theta','gamma','delta','rho','vega']

datalist4 = dict()

for data in list1:
    filter_col = [col for col in list(df) if col.startswith(data)]
    df['Port_'+ data] = 0 # initialize value, otherwise throws KeyErrorx
    for d in df[filter_col]:
        df['Port_'+ data] = df[filter_col].sum(axis=1)    

In [15]:
df.to_csv('prob.csv')